## Option 3: Load from Local OneDrive Files

Since your OneDrive files require organizational authentication, the best approach is to:

1. Make sure your OneDrive is synced locally
2. Find the local path to your synced files
3. Load them directly from your computer

### Step 1: Find Your OneDrive Files

Run this cell to search for OneDrive folders and SMIF files on your computer:

In [ ]:
import os
from pathlib import Path
import pandas as pd

# Find OneDrive folders
possible_onedrive_paths = []

# Windows paths
if os.name == 'nt':
    user_home = Path.home()
    possible_onedrive_paths.extend([
        user_home / "OneDrive",
        user_home / "OneDrive - Johns Hopkins",
        user_home / "OneDrive - Johns Hopkins University",
        user_home / "OneDrive - JHU",
    ])
    
    # Check environment variables
    if os.environ.get('OneDrive'):
        possible_onedrive_paths.append(Path(os.environ['OneDrive']))
    if os.environ.get('OneDriveCommercial'):
        possible_onedrive_paths.append(Path(os.environ['OneDriveCommercial']))

# Find existing OneDrive folders
onedrive_folders = [p for p in possible_onedrive_paths if p.exists()]

print("Found OneDrive folders:")
for folder in onedrive_folders:
    print(f"  ✓ {folder}")

# Search for SMIF files
print("\nSearching for SMIF Excel files...")

transaction_files = []
income_files = []

for folder in onedrive_folders:
    try:
        # Search for files
        for file in folder.rglob("*.xlsx"):
            if "investment_transaction" in file.name.lower():
                transaction_files.append(file)
            elif "income_and_expense" in file.name.lower():
                income_files.append(file)
    except PermissionError:
        continue

# Display found files
if transaction_files:
    print(f"\n✓ Found {len(transaction_files)} transaction file(s):")
    for i, file in enumerate(transaction_files, 1):
        print(f"  {i}. {file}")
        
if income_files:
    print(f"\n✓ Found {len(income_files)} income file(s):")
    for i, file in enumerate(income_files, 1):
        print(f"  {i}. {file}")

### Step 2: Load Your Files

If files were found above, they'll be loaded automatically. Otherwise, update the paths below:

In [ ]:
if not data_loaded:
    # If files were found automatically, use them
    if transaction_files and income_files:
        # Use the most recent files
        transaction_path = str(sorted(transaction_files)[-1])
        income_path = str(sorted(income_files)[-1])
        
        print(f"Using files:")
        print(f"  Transaction: {transaction_path}")
        print(f"  Income: {income_path}")
    else:
        # Manual path entry - UPDATE THESE WITH YOUR ACTUAL PATHS
        # Example Windows path:
        # transaction_path = r"C:\Users\YourName\OneDrive - Johns Hopkins\SMIF\Investment_Transaction_Detail_-_Customizable.xlsx"
        # income_path = r"C:\Users\YourName\OneDrive - Johns Hopkins\SMIF\Income_and_Expense_Detail_Base_by_Account.xlsx"
        
        # Example Mac path:
        # transaction_path = "/Users/YourName/OneDrive - Johns Hopkins/SMIF/Investment_Transaction_Detail_-_Customizable.xlsx"
        # income_path = "/Users/YourName/OneDrive - Johns Hopkins/SMIF/Income_and_Expense_Detail_Base_by_Account.xlsx"
        
        print("❌ No files found automatically. Please update the paths below:")
        transaction_path = r"UPDATE_THIS_PATH/Investment_Transaction_Detail_-_Customizable.xlsx"
        income_path = r"UPDATE_THIS_PATH/Income_and_Expense_Detail_Base_by_Account.xlsx"
    
    # Try to load the files
    try:
        if os.path.exists(transaction_path) and os.path.exists(income_path):
            print("\nLoading files...")
            transaction_data = pd.read_excel(transaction_path)
            income_data = pd.read_excel(income_path)
            data_loaded = True
            
            print("\n✓ Files loaded successfully!")
            print(f"  Transaction data: {len(transaction_data)} rows")
            print(f"  Income data: {len(income_data)} rows")
            
            # Show preview
            print("\nTransaction data preview:")
            print(transaction_data.head(3))
            
        else:
            print("\n❌ Files not found. Please check the paths.")
            if not os.path.exists(transaction_path):
                print(f"  Missing: {transaction_path}")
            if not os.path.exists(income_path):
                print(f"  Missing: {income_path}")
            data_loaded = False
            
    except Exception as e:
        print(f"\n❌ Error loading files: {str(e)}")
        print("\nTroubleshooting:")
        print("1. Make sure OneDrive is running and files are synced")
        print("2. Close the Excel files if they're open")
        print("3. Check that you have read permissions")
        data_loaded = False

### Alternative: Manual File Selection

If the above doesn't work, you can manually browse and select your files:

In [ ]:
if not data_loaded:
    # Try using tkinter file dialog (works in Jupyter but not Colab)
    try:
        from tkinter import filedialog
        import tkinter as tk
        
        print("Opening file browser...")
        
        # Create root window and hide it
        root = tk.Tk()
        root.withdraw()
        
        # Ask for transaction file
        print("\nPlease select the Investment Transaction Detail Excel file:")
        transaction_path = filedialog.askopenfilename(
            title="Select Investment Transaction Detail",
            filetypes=[('Excel files', '*.xlsx'), ('All files', '*.*')]
        )
        
        if transaction_path:
            print(f"✓ Selected: {transaction_path}")
            
            # Ask for income file
            print("\nPlease select the Income and Expense Detail Excel file:")
            income_path = filedialog.askopenfilename(
                title="Select Income and Expense Detail",
                filetypes=[('Excel files', '*.xlsx'), ('All files', '*.*')]
            )
            
            if income_path:
                print(f"✓ Selected: {income_path}")
                
                # Load the files
                transaction_data = pd.read_excel(transaction_path)
                income_data = pd.read_excel(income_path)
                data_loaded = True
                
                print("\n✓ Files loaded successfully!")
                print(f"  Transaction data: {len(transaction_data)} rows")
                print(f"  Income data: {len(income_data)} rows")
            else:
                print("❌ No income file selected")
        else:
            print("❌ No transaction file selected")
            
        root.destroy()
        
    except Exception as e:
        print(f"File browser not available: {str(e)}")
        print("Please use one of the manual methods above.")